# Importing Libraries

In [ ]:
import os

import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

from matplotlib import pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

!gdown 1-Zyp-JP3f9QhPKaErBkPPFNKaPS1v74u

# Data Analysis

## Importing data

In [ ]:
df = pd.read_csv('/content/Dataset.csv')
df.loc[df["Class"] == "L", "Class"] = 0
df.loc[df["Class"] == "M", "Class"] = 1
df.loc[df["Class"] == "H", "Class"] = 2
display(df)
print(df.columns)

## Columns' analysis


### Gender
Here we can see the distribution of genders in dataset

In [ ]:
ax = sns.countplot(data=df, x="gender")

we can see that there are more male record than female \
now we check how the gender affects the performance of students

In [ ]:
ax = sns.countplot(data=df, x="gender")
gender = df['gender'].unique()
gender_avg = [sum(df[df['gender'] == i].Class)/float(len(df[df['gender'] == i])) for i in gender]
ax = sns.barplot(x=gender, y=gender_avg)

so maybe female students are more intelligent!

### NationalITy

In [ ]:
ax = sns.countplot(data=df, x="NationalITy")
plt.xticks(rotation=90)

the major number of students are from Jordan and KW \
Do the nationality affect the performance?

In [ ]:
nationality = df['NationalITy'].unique()
nationality_avg = [sum(df[df['NationalITy'] == i].Class)/float(len(df[df['NationalITy'] == i])) for i in nationality]
ax = sns.barplot(x=nationality, y=nationality_avg)
plt.xticks(rotation=90)

### PlaceofBirth

In [ ]:
ax = sns.countplot(data=df, x="PlaceofBirth")
plt.xticks(rotation=90)

Same as nationality \

### StageID

In [ ]:
ax = sns.countplot(data=df, x="StageID")

### Topic

In [ ]:
sns.displot(df, x="Topic",height=5, aspect=2.5)

### Relation

In [ ]:
ax = sns.countplot(data=df, x="Relation")

In [ ]:
relation = df['Relation'].unique()
relation_avg = [sum(df[df['Relation'] == i].Class)/float(len(df[df['Relation'] == i])) for i in relation]
ax = sns.barplot(x=relation, y=relation_avg)

so students who leave with their Mum have more performance!

### raisedhands

In [ ]:
ax = sns.displot(data=df, x="raisedhands", hue="Class")

In [ ]:
sns.pairplot(df,hue='Class')

# Deep Learning Model

## Clean up the data for modeling

In [ ]:
y = np.array(df['Class'])
X = np.array(pd.get_dummies(df.iloc[:, :-1])).astype("float")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

print("Len Train Data:", len(X_train))
print("Len Test Data:", len(X_test))

input_features = X_train.shape[1]
print("number of features:", input_features)

## Create Dataset and Dataloader

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y):
    self.X = X.astype(np.float32)
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, item):
    return self.X[item], self.y[item]

BATCH_SIZE = 16

trainset = Dataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

testset = Dataset(X_test, y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE)


## Models

### Needed functions

In [ ]:
def train_ep(dataloader, model, loss_fn, optimizer, device, n_all):
  all = len(dataloader)
  losses = []
  corrects = 0
  model.train()
  n = 0
  for X, y in dataloader:

    X = X.to(device)
    y = y.to(device)

    outputs = model(X)

    _, pred = torch.max(outputs, dim=1)
    corrects += torch.sum(pred == y)

    loss = loss_fn(outputs, y)
    losses.append(loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    n += 1
    # print(f"batch {n} / {all} -- loss: {loss.item():.5f}")
  return corrects.item() / n_all  * 100, np.mean(losses)

In [ ]:
def validation_ep(dataloader, model, loss_fn, device, n_all):
  all = len(dataloader)
  losses = []
  loss = 0
  corrects = 0
  n = 0
  model.eval()
  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)

      outputs = model(X)
      _, pred = torch.max(outputs, dim=1)
      corrects += torch.sum(pred == y)

      loss = loss_fn(outputs, y)
      losses.append(loss.item())
      
      n += 1
      # print(f"batch {n} / {all} -- loss: {loss.item():.5f}")
  return (corrects.item() / n_all) * 100, np.mean(losses)

In [ ]:
def fit(model, loss_fn, optimizer, EPOCHS):
  losses_valid = []
  losses_train = []
  acc_valid = []
  acc_train = []
  best_acc = 0

  for i in range(EPOCHS):
      train_c, train_l = train_ep(trainloader, model, loss_fn, optimizer, device, len(trainset))
      print(f"Epoch {i} ------ train accuracy {train_c:.3f}    train losses {train_l:.3f}", end='')
      losses_train.append(train_l)  
      acc_train.append(train_c)  

      val_c, val_l = validation_ep(testloader, model, loss_fn, device, len(testset))
      print(f"    valid accuracy {val_c:.3f}    valid losses {val_l:.3f}")
      losses_valid.append(val_l)
      acc_valid.append(val_c)
  return losses_valid, losses_train, acc_valid, acc_train  

In [ ]:
def plotplz(filename=None):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
  # ax1.title("Training and Validation Accuracy")
  ax1.plot(acc_valid,label="val")
  ax1.plot(acc_train,label="train")
  ax1.set_xlabel("iterations")
  ax1.set_ylabel("accuracy")
  ax1.legend()

  ax2.plot(losses_valid,label="val")
  ax2.plot(losses_train,label="train")
  ax2.set_xlabel("iterations")
  ax2.set_ylabel("loss")
  ax2.legend()
  if filename:
    fig.savefig(filename)

In [ ]:
def show_metrics(model):
  y_pred = []
  y_true = []
  with torch.no_grad():
    for X, y in testloader:
      outputs = model(X.to(device))
      _, pred = torch.max(outputs, dim=1)
      y_pred.append(pred)
      y_true.append(y)
  y_pred = torch.cat(y_pred).cpu()
  y_true = torch.cat(y_true)
  precision, recall, f1score, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
  print(f"precision: {precision}\nrecall: {recall}\nf1 score: {f1score}")
  disp = ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred))
  disp.plot()
  plt.savefig('conf.png')
  return precision, recall, f1score

### 3 Hidden Layer - ReLU - Adam

In [ ]:
class NN3layer(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN3layer(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 5e-5, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 300)

In [ ]:
plotplz("3layer.png")
show_metrics(model)

### 4 Hidden layer - ReLU - Adam

In [ ]:
class NN4layer(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layer(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 5e-5, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 300)

In [ ]:
plotplz("test.png")
show_metrics(model)

### 5 Hidden Layer - ReLU - Adam

In [ ]:
class NN5layer(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN5layer(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 5e-5, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 300)

In [ ]:
plotplz("test.png")
show_metrics(model)

### 4 Hidden Layer - ReLU - Adam - Dropout

In [ ]:
class NN4layerDrop(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerDrop(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 5e-5, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("test.png")
show_metrics(model)

### 4 Hidden Layer - ReLU - SGD - Dropout

In [ ]:
class NN4layerDrop(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, self.neurons),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerDrop(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-2, 0
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("test.png")
show_metrics(model)

### 4 Hidden Layer - Sigmooid - Adam - Dropout

In [ ]:
class NN4layerDrop(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.Dropout(0.3),
        nn.Sigmoid(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Sigmoid(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Sigmoid(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Sigmoid(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerDrop(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-2, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("test.png")
show_metrics(model)

### 4 Hidden Layer - Tanh - Adam - Dropout

In [ ]:
class NN4layerDrop(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerDrop(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-4, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("4drop.png")
show_metrics(model)

### 4 Hidden Layer - ReLU - Adam - BatchNorm

In [ ]:
class NN4layerNorm(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerNorm(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-4, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("4norm.png")
show_metrics(model)

### 3 Hidden Layer - ReLU - Adam - BatchNorm

In [ ]:
class NN4layerNorm(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerNorm(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-4, 0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("4norm.png")
show_metrics(model)

### 4 Hidden Layer - ReLU - Adam WD - BatchNorm

In [ ]:
class NN4layerNorm(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerNorm(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-5, 1e-3
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("4norm.png")
show_metrics(model)

### 4 Hidden layer - Tanh - Relu - Adam - BatchNorm

In [ ]:
class NN4layerNorm(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.Tanh(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

model = NN4layerNorm(input_features, 128).to(device)
criterion = nn.CrossEntropyLoss().to(device)

LR, WEIGHT_DECAY = 1e-5, 1e-8
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)
losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 600)

In [ ]:
plotplz("4normtanh.png")
show_metrics(model)

# Cross Validation

## Model 1:

In [ ]:
BATCH_SIZE = 16

dataset = Dataset(X, y)
criterion = nn.CrossEntropyLoss().to(device)

class NN4layerDrop(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, self.neurons),
        nn.Dropout(0.3),
        nn.Tanh(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

foldperf={}

kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
  print(f"----------------- Fold {fold+1} --------------------")
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                    dataset, 
                    batch_size=BATCH_SIZE, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=BATCH_SIZE, sampler=test_subsampler)
  model = NN4layerDrop(input_features, 128).to(device)
  LR, WEIGHT_DECAY = 1e-4, 0
  optimizer = torch.optim.Adam(
      model.parameters(),
      lr=LR,
      weight_decay=WEIGHT_DECAY
  )
  losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 500)

  plotplz("model1.png")
  precision, recall, f1score = show_metrics(model)
  foldperf['fold{}'.format(fold+1)] = [sum(losses_valid)/len(losses_valid),
                                       sum(losses_train)/len(losses_train),
                                       sum(acc_valid)/len(acc_valid),
                                       sum(acc_train)/len(acc_train),
                                       precision, recall, f1score]

  print("--------------------------------------------")

In [ ]:
cv_losses_valid, cv_losses_train, cv_acc_valid, cv_acc_train = 0, 0, 0, 0
cv_precision, cv_recall, cv_f1score = 0, 0, 0
for i in range(1, 6):
  cv_losses_valid += foldperf[f"fold{i}"][0]
  cv_losses_train += foldperf[f"fold{i}"][1]
  cv_acc_valid += foldperf[f"fold{i}"][2]
  cv_acc_train += foldperf[f"fold{i}"][3]
  cv_precision += foldperf[f"fold{i}"][4]
  cv_recall += foldperf[f"fold{i}"][5]
  cv_f1score += foldperf[f"fold{i}"][6]

print("loss train:", cv_losses_train/5, "accuracy train:", cv_acc_train/5, "loss valid:", cv_losses_valid/5, "accuracy valid", cv_acc_valid/5)
print("precision:", cv_precision/5, "recall:", cv_recall/5, "f1 score:", cv_f1score/5)

## Model 2:

In [ ]:
BATCH_SIZE = 16

dataset = Dataset(X, y)
criterion = nn.CrossEntropyLoss().to(device)

class NN4layerNorm(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

foldperf={}

kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
  print(f"----------------- Fold {fold+1} --------------------")
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                    dataset, 
                    batch_size=BATCH_SIZE, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=BATCH_SIZE, sampler=test_subsampler)
  model = NN4layerNorm(input_features, 128).to(device)
  LR, WEIGHT_DECAY = 1e-4, 0
  optimizer = torch.optim.Adam(
      model.parameters(),
      lr=LR,
      weight_decay=WEIGHT_DECAY
  )
  losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 500)

  plotplz("model1.png")
  precision, recall, f1score = show_metrics(model)
  foldperf['fold{}'.format(fold+1)] = [sum(losses_valid)/len(losses_valid),
                                       sum(losses_train)/len(losses_train),
                                       sum(acc_valid)/len(acc_valid),
                                       sum(acc_train)/len(acc_train),
                                       precision, recall, f1score]

  print("--------------------------------------------")

In [ ]:
cv_losses_valid, cv_losses_train, cv_acc_valid, cv_acc_train = 0, 0, 0, 0
cv_precision, cv_recall, cv_f1score = 0, 0, 0
for i in range(1, 6):
  cv_losses_valid += foldperf[f"fold{i}"][0]
  cv_losses_train += foldperf[f"fold{i}"][1]
  cv_acc_valid += foldperf[f"fold{i}"][2]
  cv_acc_train += foldperf[f"fold{i}"][3]
  cv_precision += foldperf[f"fold{i}"][4]
  cv_recall += foldperf[f"fold{i}"][5]
  cv_f1score += foldperf[f"fold{i}"][6]

print("loss train:", cv_losses_train/5, "accuracy train:", cv_acc_train/5, "loss valid:", cv_losses_valid/5, "accuracy valid", cv_acc_valid/5)
print("precision:", cv_precision/5, "recall:", cv_recall/5, "f1 score:", cv_f1score/5)

## Model 3:

In [ ]:
BATCH_SIZE = 16

dataset = Dataset(X, y)
criterion = nn.CrossEntropyLoss().to(device)

class NN4layerNorm(nn.Module):
  def __init__(self, input_features, neurons):
    super().__init__()
    self.neurons = neurons
    self.layers = nn.Sequential(
        nn.Linear(input_features, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, self.neurons),
        nn.BatchNorm1d(self.neurons),
        nn.ReLU(),
        nn.Linear(self.neurons, 3),
    )


  def forward(self, x):
    return self.layers(x)

foldperf={}

kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
  print(f"----------------- Fold {fold+1} --------------------")
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                    dataset, 
                    batch_size=BATCH_SIZE, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=BATCH_SIZE, sampler=test_subsampler)
  model = NN4layerNorm(input_features, 128).to(device)
  LR, WEIGHT_DECAY = 1e-4, 0
  optimizer = torch.optim.Adam(
      model.parameters(),
      lr=LR,
      weight_decay=WEIGHT_DECAY
  )
  losses_valid, losses_train, acc_valid, acc_train = fit(model, criterion, optimizer, 500)

  plotplz("model1.png")
  precision, recall, f1score = show_metrics(model)
  foldperf['fold{}'.format(fold+1)] = [sum(losses_valid)/len(losses_valid),
                                       sum(losses_train)/len(losses_train),
                                       sum(acc_valid)/len(acc_valid),
                                       sum(acc_train)/len(acc_train),
                                       precision, recall, f1score]

  print("--------------------------------------------")

In [ ]:
cv_losses_valid, cv_losses_train, cv_acc_valid, cv_acc_train = 0, 0, 0, 0
cv_precision, cv_recall, cv_f1score = 0, 0, 0
for i in range(1, 6):
  cv_losses_valid += foldperf[f"fold{i}"][0]
  cv_losses_train += foldperf[f"fold{i}"][1]
  cv_acc_valid += foldperf[f"fold{i}"][2]
  cv_acc_train += foldperf[f"fold{i}"][3]
  cv_precision += foldperf[f"fold{i}"][4]
  cv_recall += foldperf[f"fold{i}"][5]
  cv_f1score += foldperf[f"fold{i}"][6]

print("loss train:", cv_losses_train/5, "accuracy train:", cv_acc_train/5, "loss valid:", cv_losses_valid/5, "accuracy valid", cv_acc_valid/5)
print("precision:", cv_precision/5, "recall:", cv_recall/5, "f1 score:", cv_f1score/5)